# 오늘의 목표
- DDL DML DCL
- DB를 이용하는 문제 입력, 풀기 서비스 개발 with streamlit
- api server 만들기
- 병원관리 데이터 구성

# DB에 문제 입력하는 프로그램 (참고용)

In [ ]:
# 버그들
# 1. 신규 그룹 내용을 입력하면 한번에 안됐습니다.
# 2. sql_group_jss, quiz_jss에서는 정상으로 동작, sql_group, quiz에서는 멈춤.
# tip. 모듈관리를 위한 요령
from mysql import connector
class Sql_quiz:
    HOST_NUM = '192.168.90.119'
    myname ='myname'
    pw = '1234'
    db = ' mydb'
    Port = '3306'
    def add_quiz_group(self,group):
        self.group = group        
        # conn =connector.connect(host=self.HOST_NUM,user=self.myname,password = self.pw,database=self.db,port = self.Port)
        cursor = self.conn.cursor()
        # query = f"insert into sql_group_jss(name) values('{self.group}');"
        query = f"insert into sql_group(name) values('{self.group}');"
        cursor.execute(query)        
        self.conn.commit()       
        # conn.close()
    def add_quiz(self,group_name,question,answer):        
        self.conn =connector.connect(host=self.HOST_NUM,user=self.myname,password = self.pw,database=self.db,port = self.Port)
        cursor = self.conn.cursor()
        # group_name이 sql_group 테이블에 있는지 확인
        # query = f'select id from sql_group_jss where name = "{group_name}";'
        query = f'select id from sql_group where name = "{group_name}";'
        # print(query)
        cursor.execute(query)
        rows = cursor.fetchall()
        if rows:
        # if len(rows)>0:
            group_id = rows[0][0]
        else:
            ret = self.add_quiz_group(group_name)
            # query = f'select id from sql_group_jss where name = "{group_name}";'
            query = f'select id from sql_group where name = "{group_name}";'
            cursor.execute(query)
            rows = cursor.fetchall()
            # print("rows",rows,group_name)
            group_id=rows[0][0]

        # 만약 없으면 sql_group에 넣고
        # 있으면 id를 받아옵니다.
        query = f"insert into quiz(sql_group,question,answer) values({group_id},'{question}','{answer}');"
        cursor.execute(query)
        self.conn.commit()
        self.conn.close()
print("__name__",__name__)
if __name__ == "__main__":
    sql_quiz = Sql_quiz()
    sql_quiz.add_quiz("데이터베이스 기본개념2",'SQL2','데이터베이스를 관리하고 조작하기 위해 특화된 질의 언어')

# __name__ db_utils
# __name__ __main__

# DDL DML DCL
- DDL : 데이터베이스 구조 정의
- DML : 데이터 조작
- DCL : 데이터 액세스 권한 제어
- 효과적인 데이터 베이스 관리를 위해 각의 명령어 들을 적절하게 사용하기

# 문제와 정답 DB에 입력하는 Streamlit Code

In [ ]:
from mysql import connector
import streamlit as st
st.title("우리가 해낸 것")
question_input = st.text_input('문제')
answer_input = st.text_input('답을 입력하세요')
if st.button("입력"):
    "입력됨"
    st.write(answer_input)
    HOST_NUM="192.168.90.119"
    user="myname"
    password="1234"
    port="3306"
    database="mydb"
    conn =connector.connect(host=HOST_NUM,user=user, password =password,database=database,port =port)

    cursor = conn.cursor()

    quiz = 'INSERT INTO quiz(sql_group,question,answer) values (1,"{}","{}");'.format(question_input, answer_input)
    print(quiz)

    cursor.execute(quiz)
    conn.commit()
    conn.close()
    # 입력하고 기존의 데이터는 없어졌으면 좋겠다
    question_input=''
    answer_input=''
    st.experimental_rerun()

# web browser : 192.168.70.21:8501
# http://192.168.90.119:8501/

# Goorm DB 끊기지 않게 사용하는 방법
- Goorm server의 사용이 없으면 연결이 끊김
- 내부적으로 프로그램을 돌려놓으면 계속 연결이 살아있음
- terminal에서 "streamlit hello"를 실행해 두면 연결이 오래감

# 왜 굳이 테이블을 여러개 만들어서 데이터를 관리해야 하는가?
## DB 정규화
- DB 정규화는 데이터베이스 설계에서 중복을 최소화하고 일관성을 유지하기 위한 프로세스입니다. 정규화는 데이터베이스 테이블의 열(column)을 분해하고 다시 조합하여 불필요한 중복을 제거합니다. 이렇게 함으로써 데이터의 일관성과 정확성을 유지할 수 있고, 데이터베이스의 성능을 향상시킬 수 있습니다.

### DB 정규화는 총 6단계로 구성됩니다.

- 제1정규화(1NF): 테이블의 컬럼이 원자적(atomic)인 값을 가지도록 분해합니다. 즉, 하나의 컬럼에 여러 개의 값이 들어가지 않도록 합니다.

- 제2정규화(2NF): 테이블의 모든 컬럼이 기본 키(primary key)에 대해서 완전 함수적 종속을 만족하도록 분해합니다. 즉, 테이블에서 기본 키 이외의 모든 컬럼이 기본 키에 의존하도록 만듭니다.

- 제3정규화(3NF): 테이블에서 이행적 함수 종속(transitive functional dependency)를 제거합니다. 즉, 한 컬럼이 다른 컬럼에 종속되어 있는 경우, 그 종속 관계를 분해합니다.

- BCNF(Boyce-Codd Normal Form): 제2정규화와 비슷하지만, 모든 결정자(candidate key)가 모든 비-결정자(non-key)에 대해 함수적 종속을 가지도록 합니다.

- 제4정규화(4NF): 테이블에서 다중 값 종속(multi-valued dependency)를 분해합니다. 즉, 한 컬럼의 값이 다른 여러 개의 컬럼의 값을 결정하는 경우, 그 종속 관계를 분해합니다.

- 제5정규화(5NF): 테이블에서 조인 종속성(join dependency)를 분해합니다. 즉, 여러 개의 테이블이 동일한 키를 공유하는 경우, 그 종속 관계를 분해합니다.

DB 정규화는 데이터베이스 설계에서 중요한 역할을 합니다. 적절한 정규화 수준을 선택하여 데이터베이스를 설계하면, 데이터의 일관성과 정확성을 유지하면서도 성능을 최적화할 수 있습니다.

# DB에 그룹, 문제, 정답을 입력하는 프로그램

In [ ]:
# db_config.py
Config = {
    'host' : '13.124.204.183',
    'port' : '59918',
    'user' : 'sub330',
    'password' : '0330',
    'database' :'subDB'
  }

In [ ]:
# st_quiz.py
import streamlit as st
from db_config import Config
from quiz import Sql_quiz
import numpy as np

st.title('[PLAYDATA] SQL QUIZ')

add_selectbox = st.sidebar.selectbox(
    "무엇을 하시고 싶으신가요?",
    ("문제제출", "문제풀기", "나의 성적 확인하기")
)


if add_selectbox =='문제제출':
    st.header('SQL QUIZ 내기')

    input_group=st.text_input("Quiz Group을 입력해주세요 💡 ")


    col1, col2 = st.columns(2)

    with col1:
        input_question=st.text_input(
            "Question :question:"
        )

    with col2:
        input_answer=st.text_input(
            "Answer :heavy_exclamation_mark:"
        )
    
    if st.button('submit'):
        st.write('새로운 문제 제출 감사합니다.')

        Sql_quiz(Config).add_quiz(input_group,input_question,input_answer)

        with st.container():
            st.write("회원님이 제출하신 문제입니다. 😆")

            st.markdown(f"""## {input_group}""",True)
    
            st.markdown(f"Question : {input_question}")
            st.markdown(f"Answer : {input_answer}")

if add_selectbox =='문제풀기':
    st.write('~~~~~~미완성~~~~~~~')
    st.write(Sql_quiz(Config).get_quiz_group())
    # tab1, tab2 = st.tabs(Sql_quiz(Config).get_quiz_group())